## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,-24.7911,-50.0119,76.06,47,5,5.23,clear sky
1,1,Port Hedland,AU,-20.3167,118.5667,82.45,42,100,10.36,overcast clouds
2,2,Tuktoyaktuk,CA,69.4541,-133.0374,-16.60,76,1,2.30,clear sky
3,3,Kenai,US,60.5544,-151.2583,-2.15,78,40,3.44,scattered clouds
4,4,Avarua,CK,-21.2078,-159.7750,80.65,74,75,5.75,broken clouds


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp= float(input('What is the minimum temperature you would like for your trip?'))
max_temp= float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?85


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &
                                      (city_data_df['Max Temp'] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,-24.7911,-50.0119,76.06,47,5,5.23,clear sky
1,1,Port Hedland,AU,-20.3167,118.5667,82.45,42,100,10.36,overcast clouds
4,4,Avarua,CK,-21.2078,-159.7750,80.65,74,75,5.75,broken clouds
5,5,Kapaa,US,22.0752,-159.3190,80.58,74,75,1.01,broken clouds
6,6,Katherine,AU,-14.4667,132.2667,82.72,74,100,9.22,overcast clouds
7,7,Avera,US,33.1940,-82.5271,71.71,55,63,3.74,broken clouds
11,11,Bubaque,GW,11.2833,-15.8333,83.57,76,51,13.85,broken clouds
15,15,Rikitea,PF,-23.1203,-134.9692,73.40,65,0,6.08,clear sky
16,16,Porto Novo,BJ,6.4965,2.6036,80.91,82,68,7.07,broken clouds
20,20,Salalah,OM,17.0151,54.0924,75.29,60,0,5.75,clear sky


In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                252
City                   252
Country                251
Lat                    252
Lng                    252
Max Temp               252
Humidity               252
Cloudiness             252
Wind Speed             252
Current Description    252
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &
                                      (city_data_df['Max Temp'] >= min_temp)].dropna()

clean_df.count()

City_ID                251
City                   251
Country                251
Lat                    251
Lng                    251
Max Temp               251
Humidity               251
Cloudiness             251
Wind Speed             251
Current Description    251
dtype: int64

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Castro,BR,76.06,clear sky,-24.7911,-50.0119,
1,Port Hedland,AU,82.45,overcast clouds,-20.3167,118.5667,
4,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,
5,Kapaa,US,80.58,broken clouds,22.0752,-159.3190,
6,Katherine,AU,82.72,overcast clouds,-14.4667,132.2667,
7,Avera,US,71.71,broken clouds,33.1940,-82.5271,
11,Bubaque,GW,83.57,broken clouds,11.2833,-15.8333,
15,Rikitea,PF,73.40,clear sky,-23.1203,-134.9692,
16,Porto Novo,BJ,80.91,broken clouds,6.4965,2.6036,
20,Salalah,OM,75.29,clear sky,17.0151,54.0924,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except(IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df['Hotel Name']!="")]
clean_hotel_df.count()

City                   231
Country                231
Max Temp               231
Current Description    231
Lat                    231
Lng                    231
Hotel Name             231
dtype: int64

In [25]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
clean_df_2 = pd.read_csv('WeatherPy_vacation.csv')

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_df_2.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df_2[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))